In [40]:
import dask
import os
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import importlib

from letkf_forecasting import analyse_results, letkf_io

In [91]:
importlib.reload(analyse_results)
importlib.reload(letkf_io)

base_folder = '/a2/uaren/travis'

decimals = 3

year = 2014

# month = 4
# day = 15
# day_name = 'translation'

# month = 5
# day = 29
# day_name = 'more_complex'

month = 4
day = 26
day_name = 'two_levels'

dates = [datetime.date(2014, month, day)]
runs = [ 'owp_opt', 'persistence', 'opt_flow', 'wrf_no_div',
        'opt_flow_with_div', 'wrf', 'radiosonde', 'wrf_mean']
# runs = ['wrf_mean']

In [92]:
save_directory = "/home2/travis/python_code/letkf_forecasting/tables/"

In [93]:
horizons = [15, 30, 45, 60]

In [94]:
legend_dict = {'opt_flow': 'Opt. Flow', 
               'opt_flow_with_div': 'Opt. Flow w/ Div.',
               'wrf_no_div': 'WRF',
               'wrf': 'WRF w/ Div.',
               'owp_opt': 'BACON',                                                                                
               'persistence': 'Persis.',
               'radiosonde': 'Radiosonde',
               'wrf_mean': 'WRF Mean',
               'ens_member': 'Ens. Member'} 

In [95]:
rmse = pd.DataFrame(index=horizons, columns=runs)
rmse.index.name = 'Horizon'
correlation = rmse.copy()
bias = rmse.copy()
for run_name in runs:
    results_folder_path = os.path.join(                                          
        base_folder,                                                               
        'results',                                                               
        f'{year:04}',                                                            
        f'{month:02}',                                                           
        f'{day:02}',                                                             
        run_name)
    results_folder_path = letkf_io.find_latest_run(results_folder_path)
    results_folder_path = os.path.join(results_folder_path, 'single_day')
    
    stat_name = 'rmse'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    rmse[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'bias'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    bias[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'correlation'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    correlation[run_name] = pd.read_hdf(file_path, stat_name)

In [96]:
peices = [rmse, correlation, bias]
combined = pd.concat(peices, axis=0,
                     keys=['RMSE', 'Corr.', 'Bias'])
combined = combined.rename(columns=legend_dict)

In [97]:
def is_empty(str):
    return str != ''

In [98]:
def format_table(text, header_num=5, footer_num=2):
    text = text.split(' ')
    text = list(filter(is_empty, text))
    text = ' '.join(text)
    split_text = text.split('\n')
    split_titles2 = split_text[2].split('&')
    split_titles3 = split_text[3].split('&')
    for num, title in enumerate(split_text[2].split('&')):
        if ' w/ Div.' in title:
            split_titles2[num] = title[:-8]
            split_titles3[num] = title[-9:]
    split_titles2 = '&'.join(split_titles2)
    split_titles3 = '&'.join(split_titles3)
    split_text[2] = split_titles2
    split_text[3] = split_titles3
    
    for line_num, line in enumerate(split_text[header_num:-footer_num - 1]):
        split_line = line.split(' ')
        if split_line[0] == 'Corr.':
            Corr = True
        elif split_line[0] != '':
            Corr = False
        num_slice = slice(4, None, 2)
        numbers_str = split_line[num_slice]
        numbers = np.array(
            split_line[num_slice],
            dtype='float')
        if Corr:
            best_num = numbers.max()
        else:
            best_num = numbers[np.abs(numbers).argmin()]
        argmins = np.where(numbers == best_num)[0]
#         numbers = list(numbers.astype('str'))
        for argmin in argmins:
            numbers_str[argmin] = '\\textbf{' + numbers_str[argmin] + '}'
        split_line[num_slice] = numbers_str
        split_text[header_num + line_num] = ' '.join(split_line)
    
#     for count in range(hor_num - 1):
#         split_line.insert(((count + 1)*run_num)*2 + 2 + count, '&')
#     split_line

    
    return '\n'.join(split_text)

In [99]:
column_format = 'll' + 'c' * len(runs)
text = combined.round(decimals=decimals).to_latex(column_format=column_format)
text2 = format_table(text)
print(text2)

\begin{tabular}{llcccccccc}
\toprule
 & & BACON & Persis. & Opt. Flow & WRF & Opt. Flow & WRF & Radiosonde & WRF Mean \\
{} & Horizon & & & & & w/ Div. & w/ Div. & & \\
\midrule
RMSE & 15 & \textbf{0.226} & 0.239 & 0.230 & 0.298 & 0.270 & 0.306 & 0.362 & 0.287 \\
 & 30 & \textbf{0.278} & 0.322 & 0.292 & 0.395 & 0.346 & 0.413 & 0.389 & 0.380 \\
 & 45 & \textbf{0.289} & 0.374 & 0.342 & 0.400 & 0.389 & 0.423 & 0.374 & 0.412 \\
 & 60 & \textbf{0.295} & 0.380 & 0.351 & 0.391 & 0.385 & 0.412 & 0.354 & 0.392 \\
Corr. & 15 & \textbf{0.756} & 0.733 & 0.752 & 0.607 & 0.674 & 0.567 & 0.396 & 0.629 \\
 & 30 & \textbf{0.632} & 0.516 & 0.615 & 0.243 & 0.487 & 0.188 & 0.237 & 0.321 \\
 & 45 & \textbf{0.600} & 0.358 & 0.482 & 0.183 & 0.364 & 0.130 & 0.278 & 0.158 \\
 & 60 & \textbf{0.603} & 0.347 & 0.491 & 0.233 & 0.394 & 0.225 & 0.402 & 0.267 \\
Bias & 15 & -0.020 & \textbf{0.009} & 0.024 & -0.100 & -0.022 & -0.084 & -0.099 & -0.079 \\
 & 30 & -0.033 & \textbf{0.010} & 0.044 & -0.115 & -0.044 & -0.07

In [100]:
this_file = os.path.join(save_directory, f'{day_name}_results.tex')
with open(this_file, 'w') as file:
    file.write(text2)